In [1]:
# Copyright 2019-2020 Jordi Corbilla. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
import numpy as np
import tensorflow as tf
from skimage import exposure


class ImageTreatment:
    def __init__(self, image_size):
        self.image_size = image_size

    def scaling(self, image, scale_vector):
        # Resize to 4-D vector
        image = np.reshape(image, (1, self.image_size, self.image_size, 3))
        boxes = np.zeros((len(scale_vector), 4), dtype=np.float32)
        for index, scale in enumerate(scale_vector):
            x1 = y1 = 0.5 - 0.5 * scale
            x2 = y2 = 0.5 + 0.5 * scale
            boxes[index] = np.array([y1, x1, y2, x2], dtype=np.float32)
        box_ind = np.zeros((len(scale_vector)), dtype=np.int32)
        crop_size = np.array([self.image_size, self.image_size], dtype=np.int32)

        output = tf.image.crop_and_resize(image, boxes, box_ind, crop_size)
        output = np.array(output, dtype=np.uint8)
        return output

    def brightness(self, image, delta):
        output = tf.image.adjust_brightness(image, delta)
        output = np.array(output, dtype=np.uint8)
        return output

    def contrast(self, image, contrast_factor):
        output = tf.image.adjust_contrast(image, contrast_factor)
        output = np.array(output, dtype=np.uint8)
        return output

    def saturation(self, image, saturation_factor):
        output = tf.image.adjust_saturation(image, saturation_factor)
        output = np.array(output, dtype=np.uint8)
        return output

    def hue(self, image, delta):
        output = tf.image.adjust_hue(image, delta)
        output = np.array(output, dtype=np.uint8)
        return output

    def central_crop(self, image, central_fraction):
        output = tf.image.central_crop(image, central_fraction)
        output = np.array(output, dtype=np.uint8)
        return output

    def crop_to_bounding_box(self, image, offset_height, offset_width, target_height, target_width):
        output = tf.image.crop_to_bounding_box(image, offset_height, offset_width, target_height, target_width)
        output = tf.image.resize(output, (self.image_size, self.image_size))
        output = np.array(output, dtype=np.uint8)
        return output

    def gamma(self, image, gamma):
        output = tf.image.adjust_gamma(image, gamma)
        output = np.array(output, dtype=np.uint8)
        return output

    def rot90(self, image, k):
        output = tf.image.rot90(image, k)
        output = np.array(output, dtype=np.uint8)
        return output

    def rescale_intensity(self, image):
        p2, p98 = np.percentile(image, (2, 98))
        img_rescale = exposure.rescale_intensity(image, in_range=(p2, p98))
        return img_rescale

    def equalize_histogram(self, image):
        img_eq = exposure.equalize_hist(image)
        return img_eq

    def equalize_adapthist(self, image):
        img_adapted = exposure.equalize_adapthist(image, clip_limit=0.03)
        return img_adapted

In [2]:
# Copyright 2019-2020 Jordi Corbilla. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
import csv
import os
import cv2
import numpy as np



class DataAugmentationStrategy:
    def __init__(self, image_size, file_name):
        self.base_image = file_name
        self.treatment = ImageTreatment(image_size)
        self.file_path = r'C:\Users\Green\Documents\ODIRCODE\PreprocessingCodes\ODIR-5K_Training_Images_treated_' + str(image_size)
        self.saving_path = r'C:\Users\Green\Documents\ODIRCODE\PreprocessingCodes\ODIR-5K_Training_Dataset_augmented_' + str(image_size)
        self.file_id = file_name.replace('.jpg', '')

    def save_image(self, original_vector, image, sample):
        black = [0, 0, 0]
        diff = 5
        boundaries = [([black[2], black[1], black[0]],
           [black[2]+diff, black[1]+diff, black[0]+diff])]
        central = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        file = self.file_id + '_'+str(sample)+'.jpg'
        file_name = os.path.join(self.saving_path, file)
        exists = os.path.isfile(file_name)
        width = int(central.shape[1])
        height = int(central.shape[0])
        Size = (width, height)
        for (lower, upper) in boundaries:

            # You get the lower and upper part of the interval:
            lower = np.array(lower, dtype=np.uint8)
            upper = np.array(upper, dtype=np.uint8)

            # cv2.inRange is used to binarize (i.e., render in white/black) an image
            # All the pixels that fall inside your interval [lower, uipper] will be white
            # All the pixels that do not fall inside this interval will
            # be rendered in black, for all three channels:
            mask = cv2.inRange(central, lower, upper)

            

            # Now, you AND the mask and the input image
            # All the pixels that are white in the mask will
            # survive the AND operation, all the black pixels
            # will remain black
            output = cv2.bitwise_and(central, central, mask=mask)

            # Check out the ANDed mask:
            #v2.imshow("ANDed mask", output)
            #v2.waitKey(0)

            # You can use the mask to count the number of white pixels.
            # Remember that the white pixels in the mask are those that
            # fall in your defined range, that is, every white pixel corresponds
            # to a green pixel. Divide by the image size and you got the
            # percentage of green pixels in the original image:
            ratio_black = cv2.countNonZero(mask)/(central.size/3)

            # This is the color percent calculation, considering the resize I did earlier.
            colorPercent = (ratio_black * 100)

            # Print the color percent, use 2 figures past the decimal point
            #print('black pixel percentage:', np.round(colorPercent, 2))
            per = np.round(colorPercent, 2)
            # numpy's hstack is used to stack two images horizontally,
            # so you see the various images generated in one figure:
            #cv2.imshow("images", np.hstack([central, output]))
            #cv2.waitKey(0)
                #number_of_all_pix = np.sum(image)
                #number_of_black_pix = np.sum(image == 0)
                #print(number_of_all_pix,number_of_black_pix)
                #if (number_of_black_pix / float(number_of_all_pix)) <= 0.1: 
            if per <= 60:
                if exists:
                    print("duplicate file found: " + file_name)
                status = cv2.imwrite(file_name, central)
                with open(r'C:\Users\Green\Documents\ODIRCODE\Prepro\ground_truth\odir_augmented.csv', 'a', newline='') as csv_file:
                    file_writer = csv.writer(csv_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                    file_writer.writerow([file, original_vector[1], original_vector[2], original_vector[3], original_vector[4],
                                           original_vector[5], original_vector[6], original_vector[7], original_vector[8]])

                #print(file_name + " written to file-system : ", status)

    def generate_images(self, number_samples, original_vector, weights):
        eye_image = os.path.join(self.file_path, self.base_image)
        image = cv2.imread(eye_image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        original_image = image
        saved = 0

        # For any repeating elements, just give the other output
        # We are only expecting up to 3 repetitions
        if weights == 20:
            original_image = self.treatment.rot90(original_image, 2)
        if weights == 400:
            original_image = self.treatment.rot90(original_image, 3)
        if weights > 401:
            print(str(self.file_id) + ' samples:' + str(number_samples))
            raise ValueError('this cannot happen')

        # for the sample type 14, just generate 1 image and leave the method
        if number_samples == 14:
            central = self.treatment.rot90(original_image, 1)
            self.save_image(original_vector, central, weights+14)
            saved = saved +1
            return saved

        if number_samples > 0:
            central = self.treatment.crop_to_bounding_box(original_image, 0, 0, 112, 112)
            self.save_image(original_vector, central, weights+0)
            saved = saved + 1

        if number_samples > 1:
            central = self.treatment.crop_to_bounding_box(original_image, 112, 0, 112, 112)
            self.save_image(original_vector, central, weights+1)
            saved = saved + 1

        if number_samples > 2:
            central = self.treatment.crop_to_bounding_box(original_image, 0, 112, 112, 112)
            self.save_image(original_vector, central, weights+2)
            saved = saved + 1

        if number_samples > 3:
            central = self.treatment.crop_to_bounding_box(original_image, 112, 112, 112, 112)
            self.save_image(original_vector, central, weights+3)
            saved = saved + 1

        if number_samples > 4:
            vector = [0.50]
            central = self.treatment.scaling(original_image, vector)
            self.save_image(original_vector, central[0], weights+4)
            saved = saved + 1

        if number_samples > 5:
            vector = [0.70]
            central = self.treatment.scaling(original_image, vector)
            self.save_image(original_vector, central[0], weights+5)
            saved = saved + 1

        if number_samples > 6:
            vector = [0.80]
            central = self.treatment.scaling(original_image, vector)
            self.save_image(original_vector, central[0], weights+6)
            saved = saved + 1

        if number_samples > 7:
            vector = [0.90]
            central = self.treatment.scaling(original_image, vector)
            self.save_image(original_vector, central[0], weights+7)
            saved = saved + 1

        if number_samples > 8:
            central = self.treatment.rescale_intensity(original_image)
            self.save_image(original_vector, central, weights+8)
            saved = saved + 1

        if number_samples > 9:
            central = self.treatment.contrast(original_image, 2)
            self.save_image(original_vector, central, weights+9)
            saved = saved + 1

        if number_samples > 10:
            central = self.treatment.saturation(original_image, 0.5)
            self.save_image(original_vector, central, weights+10)
            saved = saved + 1

        if number_samples > 11:
            central = self.treatment.gamma(original_image, 0.5)
            self.save_image(original_vector, central, weights+11)
            saved = saved + 1

        if number_samples > 12:
            central = self.treatment.hue(original_image, 0.2)
            self.save_image(original_vector, central, weights+12)
            saved = saved + 1

        return saved

In [3]:
# Copyright 2019-2020 Jordi Corbilla. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
import csv


class GroundTruthFiles:
    def __init__(self):
        self.amd = []
        self.cataract = []
        self.diabetes = []
        self.glaucoma = []
        self.hypertension = []
        self.myopia = []
        self.others = []

    def populate_vectors(self, ground_truth_file):
        with open(ground_truth_file) as csvDataFile:
            csv_reader = csv.reader(csvDataFile)

            for row in csv_reader:
                column_id = row[0]
                normal = row[1]
                diabetes = row[2]
                glaucoma = row[3]
                cataract = row[4]
                amd = row[5]
                hypertension = row[6]
                myopia = row[7]
                others = row[8]
                # just discard the first row
                if column_id != "ID":
                    print("Processing image: " + column_id + "_left.jpg")
                    if diabetes == '1':
                        self.diabetes.append([column_id, normal, diabetes, glaucoma, cataract, amd, hypertension, myopia, others])
                    if glaucoma == '1':
                        self.glaucoma.append([column_id, normal, diabetes, glaucoma, cataract, amd, hypertension, myopia, others])
                    if cataract == '1':
                        self.cataract.append([column_id, normal, diabetes, glaucoma, cataract, amd, hypertension, myopia, others])
                    if amd == '1':
                        self.amd.append([column_id, normal, diabetes, glaucoma, cataract, amd, hypertension, myopia, others])
                    if hypertension == '1':
                        self.hypertension.append([column_id, normal, diabetes, glaucoma, cataract, amd, hypertension, myopia, others])
                    if myopia == '1':
                        self.myopia.append([column_id, normal, diabetes, glaucoma, cataract, amd, hypertension, myopia, others])
                    if others == '1':
                        self.others.append([column_id, normal, diabetes, glaucoma, cataract, amd, hypertension, myopia, others])

In [4]:
# Copyright 2019-2020 Jordi Corbilla. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
import csv
import logging.config
import os
from absl import app

#from odir_data_augmentation_strategies import DataAugmentationStrategy
#from odir_load_ground_truth_files import GroundTruthFiles


def write_header():
    with open(r'C:\Users\Green\Documents\ODIRCODE\Prepro\ground_truth\odir_augmented.csv', 'w', newline='') as csv_file:
        file_writer = csv.writer(csv_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        file_writer.writerow(['ID', 'Normal', 'Diabetes', 'Glaucoma', 'Cataract', 'AMD', 'Hypertension',
                              'Myopia', 'Others'])
        return file_writer


def process_files(images, cache, files):
    total = 0
    for strategy in range(len(images)):
        images_to_process = images[strategy][0]
        samples_per_image = images[strategy][1]
        for image_index in range(images_to_process):
            #if image_index==images_to_process:
                #break
            image_vector = files[image_index]
            file_name = image_vector[0]

            # Only check during the first strategy
            if strategy == 0:
                if file_name not in cache:
                    cache[file_name] = 1
                else:
                    cache[file_name] = cache[file_name] * 20

            # print('Processing: ' + file_name)
            augment = DataAugmentationStrategy(image_size, file_name)
            count = augment.generate_images(samples_per_image, image_vector, cache[file_name])
            total = total + count
    return total




In [5]:
#def main(argv):
image_size = 448
csv_path = r'C:\Users\Green\Documents\ODIRCODE\Prepro\ground_truth\odir.csv'
# load the ground truth file
files = GroundTruthFiles()
files.populate_vectors(csv_path)

print('files record count order by size ASC')
print('hypertension ' + str(len(files.hypertension)))
print('myopia ' + str(len(files.myopia)))
print('cataract ' + str(len(files.cataract)))
print('amd ' + str(len(files.amd)))
print('glaucoma ' + str(len(files.glaucoma)))
print('others ' + str(len(files.others)))
print('diabetes ' + str(len(files.diabetes)))

images_hypertension = [[len(files.hypertension), 13], [128, 14]]
images_myopia = [[len(files.myopia), 9], [196, 14]]
images_cataract = [[len(files.cataract), 9], [66, 14]]
images_amd = [[len(files.amd), 9], [16, 14]]
images_glaucoma = [[len(files.glaucoma), 7], [312, 14]]
#images_others = [[len(files.others), 1], [1300, 14]] 
images_others = [[len(files.others), 1], [568, 14]] 
#images_diabetes = [[2500, 1]]
images_diabetes = [[1038, 1]]

# Delete previous file
exists = os.path.isfile(r'C:\Users\Green\Documents\ODIRCODE\Prepro\ground_truth\odir_augmented.csv')
if exists:
    os.remove(r'C:\Users\Green\Documents\ODIRCODE\Prepro\ground_truth\odir_augmented.csv')

write_header()

images_processed = {}

total_hypertension = process_files(images_hypertension, images_processed, files.hypertension)
total_myopia = process_files(images_myopia, images_processed, files.myopia)
total_cataract = process_files(images_cataract, images_processed, files.cataract)
total_amd = process_files(images_amd, images_processed, files.amd)
total_glaucoma = process_files(images_glaucoma, images_processed, files.glaucoma)
total_others = process_files(images_others, images_processed, files.others)
total_diabetes = process_files(images_diabetes, images_processed, files.diabetes)

print("total generated hypertension: " + str(total_hypertension))
print("total generated myopia: " + str(total_myopia))
print("total generated cataract: " + str(total_cataract))
print("total generated amd: " + str(total_amd))
print("total generated glaucoma: " + str(total_glaucoma))
print("total generated others: " + str(total_others))
print("total generated diabetes: " + str(total_diabetes))

#if __name__ == '__main__':
    # create logger
    #logging.config.fileConfig('logging.conf')
    #logger = logging.getLogger('odir')
    #image_size = 224
    #csv_path = r'C:\Users\Green\Documents\ODIRCODE\Prepro\ground_truth\odir.csv'
    #app.run(main)

Processing image: 0_left.jpg_left.jpg
Processing image: 0_right.jpg_left.jpg
Processing image: 1_left.jpg_left.jpg
Processing image: 1_right.jpg_left.jpg
Processing image: 2_left.jpg_left.jpg
Processing image: 2_right.jpg_left.jpg
Processing image: 3_left.jpg_left.jpg
Processing image: 3_right.jpg_left.jpg
Processing image: 4_left.jpg_left.jpg
Processing image: 4_right.jpg_left.jpg
Processing image: 5_left.jpg_left.jpg
Processing image: 5_right.jpg_left.jpg
Processing image: 6_left.jpg_left.jpg
Processing image: 6_right.jpg_left.jpg
Processing image: 7_left.jpg_left.jpg
Processing image: 7_right.jpg_left.jpg
Processing image: 8_left.jpg_left.jpg
Processing image: 8_right.jpg_left.jpg
Processing image: 9_left.jpg_left.jpg
Processing image: 9_right.jpg_left.jpg
Processing image: 10_left.jpg_left.jpg
Processing image: 10_right.jpg_left.jpg
Processing image: 11_left.jpg_left.jpg
Processing image: 11_right.jpg_left.jpg
Processing image: 12_left.jpg_left.jpg
Processing image: 12_right.jpg_le

Processing image: 2756_right.jpg_left.jpg
Processing image: 2757_left.jpg_left.jpg
Processing image: 2758_left.jpg_left.jpg
Processing image: 2758_right.jpg_left.jpg
Processing image: 2760_left.jpg_left.jpg
Processing image: 2760_right.jpg_left.jpg
Processing image: 2761_left.jpg_left.jpg
Processing image: 2761_right.jpg_left.jpg
Processing image: 2762_left.jpg_left.jpg
Processing image: 2762_right.jpg_left.jpg
Processing image: 2763_left.jpg_left.jpg
Processing image: 2763_right.jpg_left.jpg
Processing image: 2764_left.jpg_left.jpg
Processing image: 2764_right.jpg_left.jpg
Processing image: 2765_left.jpg_left.jpg
Processing image: 2765_right.jpg_left.jpg
Processing image: 2766_left.jpg_left.jpg
Processing image: 2766_right.jpg_left.jpg
Processing image: 2767_left.jpg_left.jpg
Processing image: 2767_right.jpg_left.jpg
Processing image: 2768_left.jpg_left.jpg
Processing image: 2768_right.jpg_left.jpg
Processing image: 2769_left.jpg_left.jpg
Processing image: 2769_right.jpg_left.jpg
Proc

total generated hypertension: 2624
total generated myopia: 2554
total generated cataract: 2541
total generated amd: 2536
total generated glaucoma: 2503
total generated others: 1692
total generated diabetes: 1038


In [6]:
#def main(argv):
image_size = 448
csv_path = r'C:\Users\Green\Documents\ODIRCODE\Prepro\ground_truth\odir_augmented.csv'
# load the ground truth file
files = GroundTruthFiles()
files.populate_vectors(csv_path)
a=str(len(files.hypertension))
b= str(len(files.myopia))
c= str(len(files.cataract))
d= str(len(files.amd))
e= str(len(files.glaucoma))
f= str(len(files.others))
g= str(len(files.diabetes))

Processing image: 11_left_2.jpg_left.jpg
Processing image: 11_left_3.jpg_left.jpg
Processing image: 11_left_4.jpg_left.jpg
Processing image: 11_left_5.jpg_left.jpg
Processing image: 11_left_6.jpg_left.jpg
Processing image: 11_left_7.jpg_left.jpg
Processing image: 11_left_8.jpg_left.jpg
Processing image: 11_left_9.jpg_left.jpg
Processing image: 11_left_10.jpg_left.jpg
Processing image: 11_left_11.jpg_left.jpg
Processing image: 11_left_12.jpg_left.jpg
Processing image: 11_left_13.jpg_left.jpg
Processing image: 11_right_2.jpg_left.jpg
Processing image: 11_right_3.jpg_left.jpg
Processing image: 11_right_4.jpg_left.jpg
Processing image: 11_right_5.jpg_left.jpg
Processing image: 11_right_6.jpg_left.jpg
Processing image: 11_right_7.jpg_left.jpg
Processing image: 11_right_8.jpg_left.jpg
Processing image: 11_right_9.jpg_left.jpg
Processing image: 11_right_10.jpg_left.jpg
Processing image: 11_right_11.jpg_left.jpg
Processing image: 11_right_12.jpg_left.jpg
Processing image: 11_right_13.jpg_left.

Processing image: 1588_right_5.jpg_left.jpg
Processing image: 1588_right_6.jpg_left.jpg
Processing image: 1588_right_7.jpg_left.jpg
Processing image: 1588_right_8.jpg_left.jpg
Processing image: 1588_right_9.jpg_left.jpg
Processing image: 1589_left_2.jpg_left.jpg
Processing image: 1589_left_3.jpg_left.jpg
Processing image: 1589_left_4.jpg_left.jpg
Processing image: 1589_left_5.jpg_left.jpg
Processing image: 1589_left_6.jpg_left.jpg
Processing image: 1589_left_7.jpg_left.jpg
Processing image: 1589_left_8.jpg_left.jpg
Processing image: 1589_left_9.jpg_left.jpg
Processing image: 1589_right_2.jpg_left.jpg
Processing image: 1589_right_3.jpg_left.jpg
Processing image: 1589_right_4.jpg_left.jpg
Processing image: 1589_right_5.jpg_left.jpg
Processing image: 1589_right_6.jpg_left.jpg
Processing image: 1589_right_7.jpg_left.jpg
Processing image: 1589_right_8.jpg_left.jpg
Processing image: 1589_right_9.jpg_left.jpg
Processing image: 1590_right_2.jpg_left.jpg
Processing image: 1590_right_3.jpg_left.

Processing image: 614_left_5.jpg_left.jpg
Processing image: 614_left_6.jpg_left.jpg
Processing image: 614_left_7.jpg_left.jpg
Processing image: 614_left_8.jpg_left.jpg
Processing image: 614_left_9.jpg_left.jpg
Processing image: 615_left_2.jpg_left.jpg
Processing image: 615_left_3.jpg_left.jpg
Processing image: 615_left_4.jpg_left.jpg
Processing image: 615_left_5.jpg_left.jpg
Processing image: 615_left_6.jpg_left.jpg
Processing image: 615_left_7.jpg_left.jpg
Processing image: 615_left_8.jpg_left.jpg
Processing image: 615_left_9.jpg_left.jpg
Processing image: 615_right_2.jpg_left.jpg
Processing image: 615_right_3.jpg_left.jpg
Processing image: 615_right_4.jpg_left.jpg
Processing image: 615_right_5.jpg_left.jpg
Processing image: 615_right_6.jpg_left.jpg
Processing image: 615_right_7.jpg_left.jpg
Processing image: 615_right_8.jpg_left.jpg
Processing image: 615_right_9.jpg_left.jpg
Processing image: 638_left_2.jpg_left.jpg
Processing image: 638_left_3.jpg_left.jpg
Processing image: 638_left

Processing image: 4072_left_20.jpg_left.jpg
Processing image: 4072_right_20.jpg_left.jpg
Processing image: 4081_left_1.jpg_left.jpg
Processing image: 4081_right_1.jpg_left.jpg
Processing image: 4099_left_1.jpg_left.jpg
Processing image: 4107_left_1.jpg_left.jpg
Processing image: 4110_left_1.jpg_left.jpg
Processing image: 4110_right_1.jpg_left.jpg
Processing image: 4129_right_20.jpg_left.jpg
Processing image: 4148_left_1.jpg_left.jpg
Processing image: 4148_right_1.jpg_left.jpg
Processing image: 4162_left_1.jpg_left.jpg
Processing image: 4162_right_1.jpg_left.jpg


In [7]:
csv_path = r'C:\Users\Green\Documents\ODIRCODE\Prepro\ground_truth\odir.csv'
# load the ground truth file
files = GroundTruthFiles()
files.populate_vectors(csv_path)
a11 = int(a) + int(str(len(files.hypertension)))
b11 = int(b) + int(str(len(files.myopia)))
c11 = int(c) + int(str(len(files.cataract)))
d11 = int(d) + int(str(len(files.amd)))
e11 = int(e) + int(str(len(files.glaucoma)))
f11 = int(f) + int(str(len(files.others)))
g11 = int(g) + int(str(len(files.diabetes)))
print('All files record count order by size ASC') #odir & aug
print('hypertension ' + str(a11))
print('myopia ' + str(b11))
print('cataract ' + str(c11))
print('amd ' + str(d11))
print('glaucoma ' + str(e11))
print('others ' + str(f11))
print('diabetes ' + str(g11))



Processing image: 0_left.jpg_left.jpg
Processing image: 0_right.jpg_left.jpg
Processing image: 1_left.jpg_left.jpg
Processing image: 1_right.jpg_left.jpg
Processing image: 2_left.jpg_left.jpg
Processing image: 2_right.jpg_left.jpg
Processing image: 3_left.jpg_left.jpg
Processing image: 3_right.jpg_left.jpg
Processing image: 4_left.jpg_left.jpg
Processing image: 4_right.jpg_left.jpg
Processing image: 5_left.jpg_left.jpg
Processing image: 5_right.jpg_left.jpg
Processing image: 6_left.jpg_left.jpg
Processing image: 6_right.jpg_left.jpg
Processing image: 7_left.jpg_left.jpg
Processing image: 7_right.jpg_left.jpg
Processing image: 8_left.jpg_left.jpg
Processing image: 8_right.jpg_left.jpg
Processing image: 9_left.jpg_left.jpg
Processing image: 9_right.jpg_left.jpg
Processing image: 10_left.jpg_left.jpg
Processing image: 10_right.jpg_left.jpg
Processing image: 11_left.jpg_left.jpg
Processing image: 11_right.jpg_left.jpg
Processing image: 12_left.jpg_left.jpg
Processing image: 12_right.jpg_le

Processing image: 2114_left.jpg_left.jpg
Processing image: 2114_right.jpg_left.jpg
Processing image: 2115_left.jpg_left.jpg
Processing image: 2115_right.jpg_left.jpg
Processing image: 2116_left.jpg_left.jpg
Processing image: 2116_right.jpg_left.jpg
Processing image: 2117_left.jpg_left.jpg
Processing image: 2117_right.jpg_left.jpg
Processing image: 2118_left.jpg_left.jpg
Processing image: 2118_right.jpg_left.jpg
Processing image: 2119_left.jpg_left.jpg
Processing image: 2120_left.jpg_left.jpg
Processing image: 2120_right.jpg_left.jpg
Processing image: 2121_left.jpg_left.jpg
Processing image: 2121_right.jpg_left.jpg
Processing image: 2122_left.jpg_left.jpg
Processing image: 2122_right.jpg_left.jpg
Processing image: 2123_left.jpg_left.jpg
Processing image: 2123_right.jpg_left.jpg
Processing image: 2124_left.jpg_left.jpg
Processing image: 2124_right.jpg_left.jpg
Processing image: 2125_left.jpg_left.jpg
Processing image: 2125_right.jpg_left.jpg
Processing image: 2126_left.jpg_left.jpg
Proce

Processing image: 2480_right.jpg_left.jpg
Processing image: 2481_left.jpg_left.jpg
Processing image: 2481_right.jpg_left.jpg
Processing image: 2482_left.jpg_left.jpg
Processing image: 2482_right.jpg_left.jpg
Processing image: 2483_right.jpg_left.jpg
Processing image: 2484_left.jpg_left.jpg
Processing image: 2484_right.jpg_left.jpg
Processing image: 2485_left.jpg_left.jpg
Processing image: 2485_right.jpg_left.jpg
Processing image: 2486_right.jpg_left.jpg
Processing image: 2487_left.jpg_left.jpg
Processing image: 2487_right.jpg_left.jpg
Processing image: 2488_left.jpg_left.jpg
Processing image: 2488_right.jpg_left.jpg
Processing image: 2489_left.jpg_left.jpg
Processing image: 2489_right.jpg_left.jpg
Processing image: 2490_left.jpg_left.jpg
Processing image: 2490_right.jpg_left.jpg
Processing image: 2492_left.jpg_left.jpg
Processing image: 2492_right.jpg_left.jpg
Processing image: 2493_left.jpg_left.jpg
Processing image: 2493_right.jpg_left.jpg
Processing image: 2495_right.jpg_left.jpg
Pr